In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
def split_function():
    train_data = 'train_triplets.txt'
    with open(train_data, 'r') as file:
        triplets = [line for line in file.readlines()]
    train_samples, val_samples = train_test_split(triplets, test_size=0.1)
    with open('val_samples.txt', 'w') as file:
        for item in val_samples:
            file.write(item)
    with open('train_samples.txt', 'w') as file:
        for item in train_samples:
            file.write(item)
    return len(train_samples)

def compute_distances(outputs):
    positive = tf.reduce_sum(tf.square(outputs[..., 0] - outputs[..., 1]), axis=1)
    negative = tf.reduce_sum(tf.square(outputs[..., 0] - outputs[..., 2]), axis=1)
    return positive, negative

def triplet_loss(_, outputs):
    positive, negative = compute_distances(outputs)
    return tf.reduce_mean(tf.math.softplus(positive - negative))


def accuracy(_, outputs):
    positive, negative = compute_distances(outputs)
    return tf.reduce_mean(tf.cast(tf.greater_equal(negative, positive), tf.float32))


def load_triplets(triplet, training):
    ids = tf.strings.split(triplet)
    image_triplet = []
    for i in range(3):
        image = tf.io.read_file('food/' + ids[i] + '.jpg')
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (224, 224))
        image = tf.keras.applications.mobilenet_v3.preprocess_input(image)
        image_triplet.append(image)
    if training:
        return tf.stack(image_triplet, axis=0), 1
    else:
        return tf.stack(image_triplet, axis=0)

def load_dataset(dataset_filename, training=True):
    dataset = tf.data.TextLineDataset(dataset_filename)
    dataset = dataset.map(lambda triplet: load_triplets(triplet, training), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset


In [3]:
Image_height = 224
Image_width = 224
epochs = 1
train_batch_size = 40
train_sample_size = split_function()

train_dataset = load_dataset('train_samples.txt')
val_dataset = load_dataset('val_samples.txt')

pretrained_model = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=(Image_height, Image_width, 3))
pretrained_model.trainable = False
custom_layers = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300),
    tf.keras.layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))
    ]) 
inputs = tf.keras.Input(shape=(3, Image_height, Image_width, 3))
output_triplet = []
for i in range(3):
    output_triplet.append(custom_layers(pretrained_model(inputs[:, i, ...])))
    
outputs = tf.stack(output_triplet, axis=-1)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
#model.summary()

train_dataset = train_dataset.shuffle(1024, reshuffle_each_iteration=True).repeat().batch(train_batch_size)
val_dataset = val_dataset.batch(train_batch_size)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=triplet_loss, metrics=[accuracy])

earlystopping = EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1)
redlrplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", factor=0.5, patience=1, verbose=2)
callbacks_list = [earlystopping, redlrplat] 

model.fit(train_dataset, steps_per_epoch=int(np.ceil(train_sample_size / train_batch_size)), callbacks = callbacks_list,
epochs=epochs, validation_data=val_dataset)

1340/1340 [==============================] - 1577s 1s/step - loss: 0.5139 - accuracy: 0.7557 - val_loss: 0.4885 - val_accuracy: 0.7826 - lr: 5.0000e-04


In [16]:
model.save_weights('model_weights.h5')

In [4]:
inference_batch_size = 60
test_sample_size = 59544
test_dataset = load_dataset('test_triplets.txt', training=False).batch(inference_batch_size).prefetch(2)

positive, negative = compute_distances(model.output)
predictions = tf.cast(tf.greater_equal(negative, positive), tf.int8)
test_model = tf.keras.Model(inputs=model.inputs, outputs=predictions)

predictions = test_model.predict(test_dataset, steps=int(np.ceil(test_sample_size / inference_batch_size)), verbose=1)

993/993 [==============================] - 1604s 2s/step


In [5]:
np.savetxt('predictions.txt', predictions, fmt='%i')